# DGI/KyFromAbove Workshop
#### Getting Started

September 22, 2025
Kentucky GIS Conference
Fun Exploration of KyFromAbove with Python and other Tools

Let's get started using a notebook.  If you are using MyBinder to run this notebook, the environment is mostly set up already.

## Some basic code

Install python modules/packages/libraries . .  whatever.  Let's make sure our code works.

*\* For future tutorials, I may use [obstore](https://developmentseed.org/obstore/) to replace the botocore libraries.*

In [1]:
import boto3                         # AWS Python SDK
from botocore import UNSIGNED        # access AWS with credentions, for
from botocore.client import Config   # open data only
from leafmap import Map              # interactive mapping in Python
from IPython.display import display  # display maps in Jupyter Notebooks
import geopandas as gpd              # working with geospatial data in Python

Let's get ready to build a map.  First, we are going to visit [The Commonwealth Map](https://kygeonet.ky.gov/tcm) to use the coordinate widget to grab center coordinates.

<p align="center">
    <img src="assets/coordinate_widget.png" width="450" height="auto" />
</p>

I was able to copy: -84.886 37.917 Degrees.  We can use this to center our map.  

Notice the values pasted are in the order of long/lat.  When we go to add the values for the map, make sure to reverse the order so that it is lat/long

```juypter
m = Map(center=(lat, long, zoom=4, width= , height=))
```

Fill the values for `center`, `zoom`, and `size`.

In [2]:
# let's create a map
m = Map(
    center=(37, -85.5), 
    zoom=7,
    # width=500, 
    # height=300
    )
# display the map by using m or display(m)
m

Map(center=[37, -85.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

AttributeError: 'Map' object has no attribute 'cog_layer_dict'

AttributeError: 'Map' object has no attribute 'cog_layer_dict'

Familiarize yourself with the map and the default widgets on the left.  Take a moment to test drive the ones on the left.  

- zoom
- fullscreen
- polyline
- polygon
- rectangle
- circle
- marker
- edit
- delete

When you are done, go to the previous cell and make adjustments to zoom, height, width, or coordinates.  

Now take a moment to open the toolbox in the top right.  Scroll over each thumbnail to see what's available.  Try them out.  We'll comeback to this later. 

While we're working on the map.  Let's add a *WMSLayer* of The Commonwealth Basemap.

In [3]:
m.add_tile_layer(
    url="https://kygisserver.ky.gov/arcgis/rest/services/WGS84WM_Services/Ky_TCM_Base_WGS84WM/MapServer/tile/{z}/{y}/{x}",
    name="TCM",
    attribution="DGI",
)
m

Map(center=[37, -85.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

___
## Get Building Heights

Building heights are an important attribute in a lot of contexts: urban studies, environmental studies, emergency management, economics, and of course, geospatial and remote sensing.

Though, using LiDAR to extract building footprints is a fun exercise, many excellent resources already exist.  I like [Overture Maps](https://docs.overturemaps.org/guides/buildings/#14/32.58453/-117.05154/0/60) building datasets. 

We are going to use overture maps dataset and KyFromAbove LiDAR Pointcloud data to extract building heights.

These are the steps we are going to take.

1. Find an area of interest
2. Get building footprint data
3. Search STAC for pointcloud data.
4. Create a Height Above Ground Model\*
5. Perform zonal statistics on buildings


\* Using the Height Above Ground model is much more efficient and accurate than using math and dsm/dtm.

### Find an AOI

To find our area of interest, let's overlay the Phase 3 Pointcloud Tile Grid on the map.

- Go to [KyFromAbove Stac-Browser](https://kygeonet.ky.gov/stac)
- Click on the *Point Cloud Phase 3 (COPC)* collection.
- Find the tile index geopackage in under *Asssets*
- Click *Copy URL*
- Paste it to your local text file or in your notebook with comments (#)<br><p><img src="assets/tileindexlink.png" width="600" height="auto" /></p>
- Add to map

In [5]:
# add tile index to map
tile_index = "https://kyfromabove.s3-us-west-2.amazonaws.com/elevation/PointCloud/TileGrids/kyfromabove_phase3_pointcloud_5k_grid.gpkg"
# read with geopandas
gdf = gpd.read_file(tile_index)
# add to map
m.add_gdf(gdf, layer_name="Phase 3 Tile Index")
m

Map(bottom=6410664.0, center=[39.02565182815265, -84.58444833755495], controls=(ZoomControl(options=['position…

In [9]:
# get AWS URL
aws_url = input("Paste the AWS S3 URL for the KyFromAbove data: ")
print(aws_url)

https://kyfromabove.s3.us-west-2.amazonaws.com/elevation/PointCloud/Phase3/N020E295_LAS_Phase3.copc.laz


### Use PDAL to create the HAG